In [ ]:
# Installing libraries
!pip install category_encoders
!pip install scikit-learn>=0.23
! pip install tensorflow
!pip install scikeras ==0.8.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 3.7 MB/s eta 0:00:00
ERROR: Invalid requirement: '==0.8.0': Expected package name at the start of dependency specifier
    ==0.8.0
    ^


In [ ]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import os
import time
from google.colab import drive
import matplotlib.pyplot as plt
import numpy as np
import calendar
import matplotlib
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import category_encoders as ce
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
import xgboost as xgb

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
import joblib

In [ ]:
# Mount the Google Drive at /content/drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load the dataset
fire_weather_terrain = pd.read_csv('/content/drive/MyDrive/0.Data science final project/5. Fire weather terrain combined/fire_weather_terrain_final.csv')


In [ ]:
fire_weather_terrain.head()


,latitude,longitude,brightness,scan,track,date,satellite,instrument,confidence,bright_t31,...,terrain_elevation,terrain_slope,terrain_aspect,wind_speed,wind_direction,wind_direction_category,elevation_bin,slope_category,aspect_direction,wind_speed_category
0,48.62248,-123.53745,315.41,0.39,0.36,2024-03-12,SUOMI,VIIRS,n,276.92,...,141.053091,17.995291,277.745056,1.851432,262.610761,SW,0-300,Moderate,W,Low
1,48.46122,-124.04675,336.03,0.39,0.36,2024-03-15,SUOMI,VIIRS,n,289.20,...,499.959241,11.635393,189.708092,2.413118,17.410029,N,301-600,Moderate,S,Medium
2,48.46284,-124.04449,331.22,0.39,0.36,2024-03-15,SUOMI,VIIRS,n,290.66,...,534.427533,10.487661,189.553949,2.447986,17.316382,N,301-600,Moderate,S,Medium
3,47.21657,-123.03061,326.83,0.39,0.36,2024-03-18,SUOMI,VIIRS,n,278.20,...,41.047428,1.491172,29.854566,1.054044,104.283876,E,0-300,Gentle,NE,Low
4,48.45983,-124.04285,333.52,0.47,0.48,2024-03-18,SUOMI,VIIRS,n,291.09,...,481.502214,7.948330,237.428700,0.552493,355.094673,NW,301-600,Gentle,SW,Low


In [ ]:
fire_weather_terrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 465195 entries, 0 to 465194
Data columns (total 28 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   latitude                 465195 non-null  float64
 1   longitude                465195 non-null  float64
 2   brightness               465195 non-null  float64
 3   scan                     465195 non-null  float64
 4   track                    465195 non-null  float64
 5   date                     465195 non-null  object 
 6   satellite                465195 non-null  object 
 7   instrument               465195 non-null  object 
 8   confidence               465195 non-null  object 
 9   bright_t31               465195 non-null  float64
 10  frp                      465195 non-null  float64
 11  daynight                 465195 non-null  object 
 12  type                     465195 non-null  object 
 13  frp_brightness_ratio     465195 non-null  float64
 14  day_

In [ ]:

# Create a copy of the dataset - this dataset will be used to encode and apply scaler
fire_weather_terrain_cleaned = fire_weather_terrain.copy()

In [ ]:
# Drop columns which have been given non numerical values based on numericla columns, such as wind speed category
fire_weather_terrain_cleaned = fire_weather_terrain_cleaned.drop(['wind_direction_category', 'elevation_bin', 'slope_category', 'aspect_direction', 'wind_speed_category'], axis=1)

In [ ]:
fire_weather_terrain_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 465195 entries, 0 to 465194
Data columns (total 23 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   latitude              465195 non-null  float64
 1   longitude             465195 non-null  float64
 2   brightness            465195 non-null  float64
 3   scan                  465195 non-null  float64
 4   track                 465195 non-null  float64
 5   date                  465195 non-null  object 
 6   satellite             465195 non-null  object 
 7   instrument            465195 non-null  object 
 8   confidence            465195 non-null  object 
 9   bright_t31            465195 non-null  float64
 10  frp                   465195 non-null  float64
 11  daynight              465195 non-null  object 
 12  type                  465195 non-null  object 
 13  frp_brightness_ratio  465195 non-null  float64
 14  day_of_week           465195 non-null  object 
 15  

In [ ]:
# Encode the following columns: confidence, daynight, type, day of week, satellite, instrument
# Binary encoding day night column
fire_weather_terrain_cleaned['daynight'] = fire_weather_terrain_cleaned['daynight'].map({'D': 0, 'N': 1})


In [ ]:
print(fire_weather_terrain_cleaned['confidence'].unique())

['n' 'h' 'l' '0']


In [ ]:
# Count up the values in the confidence column
confidence_counts = fire_weather_terrain_cleaned['confidence'].value_counts()
print(confidence_counts)

confidence
n    394520
h     47963
l     22409
0       303
Name: count, dtype: int64


In [ ]:
# Replace all occurrences of the value '0' in the 'confidence' column with 'low'
fire_weather_terrain_cleaned['confidence'] = fire_weather_terrain_cleaned['confidence'].replace('0', 'l')

# Count the values again
confidence_counts_updated = fire_weather_terrain_cleaned['confidence'].value_counts()
print(confidence_counts_updated)

confidence
n    394520
h     47963
l     22712
Name: count, dtype: int64


In [ ]:
# One hot encoding
# Confidence column
fire_weather_terrain_cleaned = pd.get_dummies(fire_weather_terrain_cleaned, columns=['confidence'])

In [ ]:
# Type column
fire_weather_terrain_cleaned = pd.get_dummies(fire_weather_terrain_cleaned, columns=['type'])

In [ ]:
# Day of the week
fire_weather_terrain_cleaned = pd.get_dummies(fire_weather_terrain_cleaned, columns=['day_of_week'])

In [ ]:
# satellite
fire_weather_terrain_cleaned = pd.get_dummies(fire_weather_terrain_cleaned, columns=['satellite'])
# instrument
fire_weather_terrain_cleaned = pd.get_dummies(fire_weather_terrain_cleaned, columns=['instrument'])

In [ ]:
fire_weather_terrain_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 465195 entries, 0 to 465194
Data columns (total 39 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   latitude                       465195 non-null  float64
 1   longitude                      465195 non-null  float64
 2   brightness                     465195 non-null  float64
 3   scan                           465195 non-null  float64
 4   track                          465195 non-null  float64
 5   date                           465195 non-null  object 
 6   bright_t31                     465195 non-null  float64
 7   frp                            465195 non-null  float64
 8   daynight                       465195 non-null  int64  
 9   frp_brightness_ratio           465195 non-null  float64
 10  time                           465195 non-null  int64  
 11  scaled_date                    465195 non-null  float64
 12  era5_temperature              

In [ ]:
# Convert the one hot encoded column labels to 1/0
for col in fire_weather_terrain_cleaned.columns:
    if fire_weather_terrain_cleaned[col].dtype == 'bool':
        fire_weather_terrain_cleaned[col] = fire_weather_terrain_cleaned[col].astype(int)

In [ ]:
# Drop date, scaled date, frp brightness ratio, and bright t31 columns
# date and scaled date are now redundant as they have already been used to perform IWD
# FRP brightness ratio can be dropped as FRP and brightness will now be my target variables
# bright t31 can also be dropped due to already having the brightness column... bright t31 is redundant
fire_weather_terrain_cleaned = fire_weather_terrain_cleaned.drop(['date', 'scaled_date', 'frp_brightness_ratio', 'bright_t31'], axis=1)

In [ ]:
fire_weather_terrain_cleaned.head()

,latitude,longitude,brightness,scan,track,frp,daynight,time,era5_temperature,terrain_elevation,...,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday,satellite_Aqua,satellite_NOAA-20,satellite_NOAA-21,satellite_SUOMI,satellite_Terra,instrument_MODIS,instrument_VIIRS
0,48.62248,-123.53745,315.41,0.39,0.36,1.64,1,10,278.236472,141.053091,...,0,1,0,0,0,0,1,0,0,1
1,48.46122,-124.04675,336.03,0.39,0.36,4.36,0,20,282.733589,499.959241,...,0,0,0,0,0,0,1,0,0,1
2,48.46284,-124.04449,331.22,0.39,0.36,3.60,0,20,282.728811,534.427533,...,0,0,0,0,0,0,1,0,0,1
3,47.21657,-123.03061,326.83,0.39,0.36,2.30,1,10,279.411684,41.047428,...,0,0,0,0,0,0,1,0,0,1
4,48.45983,-124.04285,333.52,0.47,0.48,3.15,0,21,283.462244,481.502214,...,0,0,0,0,0,0,1,0,0,1


In [ ]:
# Standard scaler
# define the target columns
target_column = ['frp', 'brightness']

In [ ]:
# Separate the column from the main dataset
features_to_scale = fire_weather_terrain_cleaned.drop(target_column, axis=1)

In [ ]:
features_to_scale.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 465195 entries, 0 to 465194
Data columns (total 33 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   latitude                       465195 non-null  float64
 1   longitude                      465195 non-null  float64
 2   scan                           465195 non-null  float64
 3   track                          465195 non-null  float64
 4   daynight                       465195 non-null  int64  
 5   time                           465195 non-null  int64  
 6   era5_temperature               465195 non-null  float64
 7   terrain_elevation              465195 non-null  float64
 8   terrain_slope                  465195 non-null  float64
 9   terrain_aspect                 465195 non-null  float64
 10  wind_speed                     465195 non-null  float64
 11  wind_direction                 465195 non-null  float64
 12  confidence_h                  

In [ ]:
# Define x and y variables
x  = features_to_scale
y = fire_weather_terrain_cleaned[target_column]

In [ ]:
# Split the data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
# Initialise the scaler
scaler = StandardScaler()

In [ ]:
# Fit the scaler on the training features
scaler.fit(x_train)

# transform the training and test features
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

# convert the arrays back into a dataframe
x_train_scaled = pd.DataFrame(x_train_scaled, columns=x_train.columns)
x_test_scaled = pd.DataFrame(x_test_scaled, columns=x_test.columns)

In [ ]:
x_test_scaled.head()

,latitude,longitude,scan,track,daynight,time,era5_temperature,terrain_elevation,terrain_slope,terrain_aspect,...,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday,satellite_Aqua,satellite_NOAA-20,satellite_NOAA-21,satellite_SUOMI,satellite_Terra,instrument_MODIS,instrument_VIIRS
0,0.531157,1.187395,-0.353936,-0.381777,0.833584,-0.853354,-1.025090,1.010904,0.568134,0.190869,...,-0.406575,-0.436606,-0.440848,-0.198501,-0.700022,-0.677905,1.591857,-0.191553,-0.281249,0.281249
1,0.385657,-0.716217,0.130261,-0.039827,-1.199640,1.302303,0.751128,0.281415,-1.249296,1.284870,...,-0.406575,2.290394,-0.440848,-0.198501,-0.700022,-0.677905,1.591857,-0.191553,-0.281249,0.281249
2,1.098115,-0.689389,-0.150063,-0.210802,0.833584,-0.853354,-0.630792,-0.470095,0.375712,-0.530305,...,2.459568,-0.436606,-0.440848,-0.198501,-0.700022,1.475132,-0.628197,-0.191553,-0.281249,0.281249
3,-1.675556,0.501621,0.104777,-0.039827,0.833584,-0.673716,-1.553217,-0.574660,-0.340268,0.876894,...,-0.406575,-0.436606,-0.440848,-0.198501,-0.700022,1.475132,-0.628197,-0.191553,-0.281249,0.281249
4,0.714805,1.452867,-0.048127,-0.510009,-1.199640,1.122665,0.046052,1.035482,0.968280,-0.742978,...,-0.406575,-0.436606,-0.440848,-0.198501,-0.700022,1.475132,-0.628197,-0.191553,-0.281249,0.281249


In [ ]:
# Model evaluation
# LinReg
# Polynomial LinReg
# Decision Tree
# Random Forest
# Light GBM
# XGBoost
# ANN

# the models above were evaluated in this order. Random Forest, XGBoost and ANN were further tuned. The best performing model was a tuned version of XGBoost. This model will appear at the top, and will be saved

In [ ]:
# Parameter tuning for XGBoost 1
# Define parameter distribution
param_distributions_xgb = {
    'n_estimators': [500, 1000, 1500, 2500],  # Number of boosting rounds
    'max_depth': [ 3, 6, 9, 12],  # Maximum depth of trees
    'learning_rate': [0.01,0.02, 0.05, 0.1],  # Learning rate
    'subsample': [0.6, 0.7, 0.8 ],  # Subsample ratio of training instances
    'colsample_bytree': [0.6, 0.8],  # Subsample ratio of columns
    'gamma': [0, 0.1, 0.2, 0.5],  # Minimum loss reduction
    'reg_alpha': [0, 1, 5, 10],  # L1 regularisation term
    'reg_lambda': [0, 3, 5]   # L2 regularisation term
}

# initialise XGBoost regression
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Create Random Search CV
random_search_xgb = RandomizedSearchCV(xgb_model,
                                       param_distributions=param_distributions_xgb,
                                       n_iter=10,  # Number of random combinations to try, change as needed
                                       cv=2,
                                       scoring='neg_mean_squared_error',
                                       verbose=2,
                                       random_state=42,
                                       n_jobs=-1)

# Fit random search CV
print("Starting RandomizedSearchCV for XGBoost...")
start_time_xgb = time.time()  # Start time

random_search_xgb.fit(x_train_scaled, y_train)

end_time_xgb = time.time()  # End time
print("RandomizedSearchCV for XGBoost fitting completed in:", end_time_xgb - start_time_xgb, "seconds")


# Get the best parameters
best_params_xgb = random_search_xgb.best_params_
print("Best parameters for XGBoost:", best_params_xgb)

# Get the best estimator (XGBoost model with best parameters)
best_xgb_model = random_search_xgb.best_estimator_
print("Best estimator:", best_xgb_model)

# 5. Multi-Target Model Evaluation (using the best estimator) and convert to DF
y_pred_xgb = best_xgb_model.predict(x_test_scaled)
y_pred_xgb_df = pd.DataFrame(y_pred_xgb, columns=['frp_pred', 'brightness_pred'], index=y_test.index)


# Calculate evaluation metrics for each target
rmse_frp_xgb = mean_squared_error(y_test['frp'], y_pred_xgb_df['frp_pred'])
rmse_brightness_xgb = mean_squared_error(y_test['brightness'], y_pred_xgb_df['brightness_pred'])
r2_frp_xgb = r2_score(y_test['frp'], y_pred_xgb_df['frp_pred'])
r2_brightness_xgb = r2_score(y_test['brightness'], y_pred_xgb_df['brightness_pred'])

# Print the evaluation metrics
print("--- Multi-Target XGBoost Regression Results (with RandomisedSearchCV) ---")
print("RMSE for FRP:", rmse_frp_xgb)
print("RMSE for Brightness:", rmse_brightness_xgb)
print("R-squared for FRP:", r2_frp_xgb)
print("R-squared for Brightness:", r2_brightness_xgb)

Starting RandomizedSearchCV for XGBoost...
Fitting 2 folds for each of 10 candidates, totalling 20 fits
RandomizedSearchCV for XGBoost fitting completed in: 639.0672733783722 seconds
Best parameters for XGBoost: {'subsample': 0.7, 'reg_lambda': 0, 'reg_alpha': 0, 'n_estimators': 1500, 'max_depth': 12, 'learning_rate': 0.05, 'gamma': 0.2, 'colsample_bytree': 0.6}
Best estimator: XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.6, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.2, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=12, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
   

In [ ]:
# Save the model and scaler
# Save the scaler and the model and save into drive
model_XGB1_dir = '/content/drive/MyDrive/0.Data science final project/6. Model/'
os.makedirs(model_XGB1_dir, exist_ok=True)

# Save the scaler
scaler_save_path = os.path.join(model_XGB1_dir, 'scaler.joblib')
joblib.dump(scaler, scaler_save_path)

# Save model
model_save_path = os.path.join(model_XGB1_dir, 'xgb1_model.joblib')
joblib.dump(best_xgb_model, model_save_path)



['/content/drive/MyDrive/0.Data science final project/6. Model/xgb1_model.joblib']

In [ ]:
# Start by creating a simple linear regression algorithm
lin_reg = LinearRegression()
multi_lin_reg = MultiOutputRegressor(lin_reg)
multi_lin_reg.fit(x_train_scaled, y_train)

MultiOutputRegressor(estimator=LinearRegression())

In [ ]:
# Evaluate model
y_pred_linear = multi_lin_reg.predict(x_test_scaled)
y_pred_linear_df = pd.DataFrame(y_pred_linear, columns=['frp_pred', 'brightness_pred'], index=y_test.index)

In [ ]:
# Evaluate metrics

rmse_frp_linear = mean_squared_error(y_test['frp'], y_pred_linear_df['frp_pred'])
rmse_brightness_linear = mean_squared_error(y_test['brightness'], y_pred_linear_df['brightness_pred'])
r2_frp_linear = r2_score(y_test['frp'], y_pred_linear_df['frp_pred'])
r2_brightness_linear = r2_score(y_test['brightness'], y_pred_linear_df['brightness_pred'])

print("--- Multi-Target Linear Regression Results ---")
print("RMSE for FRP:", rmse_frp_linear)
print("RMSE for Brightness:", rmse_brightness_linear)
print("R-squared for FRP:", r2_frp_linear)
print("R-squared for Brightness:", r2_brightness_linear)

# Lin reg model is not complex enough
# Model is unable to capture linear relationships very well

--- Multi-Target Linear Regression Results ---
RMSE for FRP: 7838.342635409552
RMSE for Brightness: 215.14846754029824
R-squared for FRP: 0.1551866883141626
R-squared for Brightness: 0.5764872442571747


In [ ]:
# Polynomial regression to capture non linear relationships
# Degrees refers to the highest power to which the independent variables are raised. IE how complex is the curve that the model fits the data
degree = 2 # Change number of degrees
poly = PolynomialFeatures(degree=degree)
x_train_poly = poly.fit_transform(x_train_scaled)
x_test_poly = poly.transform(x_test_scaled)

# Apply multi output lin reg on polynomial features
poly_lin_reg = MultiOutputRegressor(LinearRegression())
poly_lin_reg.fit(x_train_poly, y_train)

MultiOutputRegressor(estimator=LinearRegression())

In [ ]:
# Evaluate model
y_pred_poly = poly_lin_reg.predict(x_test_poly)
y_pred_poly_df = pd.DataFrame(y_pred_poly, columns=['frp_pred', 'brightness_pred'], index=y_test.index)

In [ ]:
# Evaluate metrics
rmse_frp_poly = mean_squared_error(y_test['frp'], y_pred_poly_df['frp_pred'])
rmse_brightness_poly = mean_squared_error(y_test['brightness'], y_pred_poly_df['brightness_pred'])
r2_frp_poly = r2_score(y_test['frp'], y_pred_poly_df['frp_pred'])
r2_brightness_poly = r2_score(y_test['brightness'], y_pred_poly_df['brightness_pred'])

print("--- Multi-Target Polynomial Regression (Degree{}) Results ---".format(degree))
print("RMSE for FRP:", rmse_frp_poly)
print("RMSE for Brightness:", rmse_brightness_poly)
print("R-squared for FRP:", r2_frp_poly)
print("R-squared for Brightness:", r2_brightness_poly)

--- Multi-Target Polynomial Regression (Degree2) Results ---
RMSE for FRP: 7209.60501233748
RMSE for Brightness: 197.79689337865625
R-squared for FRP: 0.22295176803005412
R-squared for Brightness: 0.610643253238724


In [ ]:
# Decision Tree
dec_tree = DecisionTreeRegressor(random_state=42)
multi_dec_tree = MultiOutputRegressor(dec_tree)
multi_dec_tree.fit(x_train_scaled, y_train)

MultiOutputRegressor(estimator=DecisionTreeRegressor(random_state=42))

In [ ]:
y_pred_dec_tree = multi_dec_tree.predict(x_test_scaled)
y_pred_dec_tree_df = pd.DataFrame(y_pred_dec_tree, columns=['frp_pred', 'brightness_pred'], index=y_test.index)

In [ ]:
# Evaluate metrics
rmse_frp_dt = mean_squared_error(y_test['frp'], y_pred_dec_tree_df['frp_pred'])
rmse_brightness_dt = mean_squared_error(y_test['brightness'], y_pred_dec_tree_df['brightness_pred'])
r2_frp_dt = r2_score(y_test['frp'], y_pred_dec_tree_df['frp_pred'])
r2_brightness_dt = r2_score(y_test['brightness'], y_pred_dec_tree_df['brightness_pred'])

print("--- Multi-Target Decision Tree Regression Results ---")
print("RMSE for FRP:", rmse_frp_dt)
print("RMSE for Brightness:", rmse_brightness_dt)
print("R-squared for FRP:", r2_frp_dt)
print("R-squared for Brightness:", r2_brightness_dt)

--- Multi-Target Decision Tree Regression Results ---
RMSE for FRP: 9672.25860321478
RMSE for Brightness: 258.1712927041348
R-squared for FRP: -0.04247201253364996
R-squared for Brightness: 0.4917982132206632


In [ ]:
# Random forest
rand_forest = RandomForestRegressor(random_state=42, n_estimators=100)
multi_rand_forest = MultiOutputRegressor(rand_forest)
multi_rand_forest.fit(x_train_scaled, y_train)

MultiOutputRegressor(estimator=RandomForestRegressor(random_state=42))

In [ ]:
y_pred_rf = multi_rand_forest.predict(x_test_scaled)
y_pred_rf_df = pd.DataFrame(y_pred_rf, columns=['frp_pred', 'brightness_pred'], index=y_test.index)

In [ ]:
# Evaluate metrics
rmse_frp_rf = mean_squared_error(y_test['frp'], y_pred_rf_df['frp_pred'])
rmse_brightness_rf = mean_squared_error(y_test['brightness'], y_pred_rf_df['brightness_pred'])
r2_frp_rf = r2_score(y_test['frp'], y_pred_rf_df['frp_pred'])
r2_brightness_rf = r2_score(y_test['brightness'], y_pred_rf_df['brightness_pred'])

print("--- Multi-Target Random Forest Regression Results ---")
print("RMSE for FRP:", rmse_frp_rf)
print("RMSE for Brightness:", rmse_brightness_rf)
print("R-squared for FRP:", r2_frp_rf)
print("R-squared for Brightness:", r2_brightness_rf)

--- Multi-Target Random Forest Regression Results ---
RMSE for FRP: 5224.563814643959
RMSE for Brightness: 138.51854686797137
R-squared for FRP: 0.43689868334867266
R-squared for Brightness: 0.7273307489649761


In [ ]:
# Light Gradient Boost Machine
# LightGBM uses its own dataset for efficiency and speed

#Create a model for FRP
lgbm_train_data_frp = lgb.Dataset(x_train_scaled, label=y_train['frp'])
lgbm_test_data_frp = lgb.Dataset(x_test_scaled, label=y_test['frp'], reference=lgbm_train_data)

# Define parameters - use gridsearchCV to fine tune parameters afetr initial model has run
params_frp={
    'objective': 'regression_L2', #MSE loss
    'metric': 'rmse', #RMSE as evaluation metric
    'num_leaves' :31, #number of leaves per tree
    'learning_rate': 0.05, #learning rate
    'feature_fraction': 0.9, #subsample ratio of columns
    'bagging_fraction': 0.8, #subsample ratio of data
    'bagging_freq': 5, #frequency of bagging
    'verbose': -1, #suppress verbose output
    'seed':42 #random seed for reproducibility
    }

num_round = 100 #number of boosting rounds

lgbm_model_frp = lgb.train(params_frp,
                           lgbm_train_data_frp,
                           num_boost_round=num_round,
                           valid_sets=[lgbm_test_data_frp],
                           callbacks=[lgb.early_stopping(stopping_rounds=10)])

# Create a model for brightness
lgbm_train_data_brightness = lgb.Dataset(x_train_scaled, label=y_train['brightness'])
lgbm_test_data_brightness = lgb.Dataset(x_test_scaled, label=y_test['brightness'], reference=lgbm_train_data)

# Define parameters - use gridsearchCV to fine tune parameters afetr initial model has run
params_brightness={
    'objective': 'regression_L2', #MSE loss
    'metric': 'rmse', #RMSE as evaluation metric
    'num_leaves' :31, #number of leaves per tree
    'learning_rate': 0.05, #learning rate
    'feature_fraction': 0.9, #subsample ratio of columns
    'bagging_fraction': 0.8, #subsample ratio of data
    'bagging_freq': 5, #frequency of bagging
    'verbose': -1, #suppress verbose output
    'seed':42 #random seed for reproducibility
    }

num_round = 100 #number of boosting rounds

lgbm_model_brightness = lgb.train(params_brightness,
                           lgbm_train_data_brightness,
                           num_boost_round=num_round,
                           valid_sets=[lgbm_test_data_brightness],
                           callbacks=[lgb.early_stopping(stopping_rounds=10)])



Training until validation scores don't improve for 10 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 75.906
Training until validation scores don't improve for 10 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 13.5707


In [ ]:
# Predict variables
y_pred_lgbm_frp = lgbm_model_frp.predict(x_test_scaled)
# Predict variables
y_pred_lgbm_frp = lgbm_model_frp.predict(x_test_scaled)
y_pred_lgbm_brightness = lgbm_model_brightness.predict(x_test_scaled)
# convert to dataframe
y_pred_lgbm_df = pd.DataFrame({'frp_pred' : y_pred_lgbm_frp, 'brightness_pred' : y_pred_lgbm_brightness}, index=y_test.index)


In [ ]:
# Evaluate and print metrics
rmse_frp_lgbm = mean_squared_error(y_test['frp'], y_pred_lgbm_df['frp_pred'])
rmse_brightness_lgbm = mean_squared_error(y_test['brightness'], y_pred_lgbm_df['brightness_pred'])
r2_frp_lgbm = r2_score(y_test['frp'], y_pred_lgbm_df['frp_pred'])
r2_brightness_lgbm = r2_score(y_test['brightness'], y_pred_lgbm_df['brightness_pred'])

print("--- Multi-Target LightGBM Regression Results ---")
print("RMSE for FRP:", rmse_frp_lgbm)
print("RMSE for Brightness:", rmse_brightness_lgbm)
print("R-squared for FRP:", r2_frp_lgbm)
print("R-squared for Brightness:", r2_brightness_lgbm)

--- Multi-Target LightGBM Regression Results ---
RMSE for FRP: 5761.721989747693
RMSE for Brightness: 184.16354158170603
R-squared for FRP: 0.3790039984750512
R-squared for Brightness: 0.637480062515352


In [ ]:
# XG Boost
xgb_model = xgb.XGBRegressor(n_estimators=100, random_state=42)
xgb_model.fit(x_train_scaled, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [ ]:
# Model evaluation
y_pred_xgb = xgb_model.predict(x_test_scaled)
y_pred_xgb_df = pd.DataFrame(y_pred_xgb, columns=['frp_pred', 'brightness_pred'], index=y_test.index)

In [ ]:
# Measure and print metrics
rmse_frp_xgb = mean_squared_error(y_test['frp'], y_pred_xgb_df['frp_pred'])
rmse_brightness_xgb = mean_squared_error(y_test['brightness'], y_pred_xgb_df['brightness_pred'])
r2_frp_xgb = r2_score(y_test['frp'], y_pred_xgb_df['frp_pred'])
r2_brightness_xgb = r2_score(y_test['brightness'], y_pred_xgb_df['brightness_pred'])

print("--- Multi-Target XGBoost Regression Results ---")
print("RMSE for FRP:", rmse_frp_xgb)
print("RMSE for Brightness:", rmse_brightness_xgb)
print("R-squared for FRP:", r2_frp_xgb)
print("R-squared for brightness:", r2_brightness_xgb)

--- Multi-Target XGBoost Regression Results ---
RMSE for FRP: 5746.054766245711
RMSE for Brightness: 169.49063331072537
R-squared for FRP: 0.38069260531290994
R-squared for brightness: 0.6663632048758281


In [ ]:
# Parameter tuning for Random Forest Regression
# Define the parameter distributions
param_distributions_rf = {
    'n_estimators': [100, 200, 500, 1000, 2000],  # More trees
    'max_depth': [5, 10, 15, 20, 50],  # Deeper trees
    'min_samples_split': [2, 5, 10, 20],  # More options for splits
    'min_samples_leaf': [1, 2, 4, 8],  # More options for leaves
    'max_features': ['sqrt', 'log2']  # Feature selection strategy
}

# Create a RandomForestRegressor
rf_model = RandomForestRegressor(random_state=42)

# Create RandomizedSearchCV (more efficient for large datasets)
random_search_rf = RandomizedSearchCV(rf_model,
                                       param_distributions=param_distributions_rf,
                                       n_iter=6,  # Number of random combinations to try, adjust as needed
                                       cv=3, #cross validation folds
                                       scoring='neg_mean_squared_error',
                                       verbose=2,
                                       random_state=42,
                                       n_jobs = -1)

# Fit RandomizedSearchCV
print("Starting RandomizedSearchCV...")
start_time = time.time()  # Start time

random_search_rf.fit(x_train_scaled, y_train)

end_time = time.time()  # End time
print("RandomizedSearchCV fitting completed in:", end_time - start_time, "seconds")

# Get the best parameters
best_params_rf = random_search_rf.best_params_
print("Best parameters:", best_params_rf)

# Get the best estimator (Random Forest model with best parameters)
best_rf_model = random_search_rf.best_estimator_
print("Best estimator:", best_rf_model)

# 5. Multi-Target Model Evaluation (using the best estimator) and convert to pd df
y_pred_rf = best_rf_model.predict(x_test_scaled)
y_pred_rf_df = pd.DataFrame(y_pred_rf, columns=['frp_pred', 'brightness_pred'], index=y_test.index)

# Calculate evaluation metrics for each target
rmse_frp_rf = mean_squared_error(y_test['frp'], y_pred_rf_df['frp_pred'])
rmse_brightness_rf = mean_squared_error(y_test['brightness'], y_pred_rf_df['brightness_pred'])
r2_frp_rf = r2_score(y_test['frp'], y_pred_rf_df['frp_pred'])
r2_brightness_rf = r2_score(y_test['brightness'], y_pred_rf_df['brightness_pred'])

# Print the evaluation metrics
print("--- Multi-Target Random Forest Regression Results (with RandomizedSearchCV) ---")
print("RMSE for FRP:", rmse_frp_rf)
print("RMSE for Brightness:", rmse_brightness_rf)
print("R-squared for FRP:", r2_frp_rf)
print("R-squared for Brightness:", r2_brightness_rf)

Starting RandomizedSearchCV...
Fitting 3 folds for each of 6 candidates, totalling 18 fits
RandomizedSearchCV fitting completed in: 531.5384774208069 seconds
Best parameters: {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 8, 'max_features': 'sqrt', 'max_depth': 50}
Best estimator: RandomForestRegressor(max_depth=50, max_features='sqrt', min_samples_leaf=8,
                      random_state=42)
--- Multi-Target Random Forest Regression Results (with RandomizedSearchCV) ---
RMSE for FRP: 5466.992997144273
RMSE for Brightness: 156.87355947050023
R-squared for FRP: 0.4107697668106065
R-squared for Brightness: 0.6911995040722592


In [ ]:
# Parameter tuning for XGBoost 2
# Define parameter distribution
param_distributions_xgb = {
    'n_estimators': [500, 1000, 1500, 2500],  # Number of boosting rounds
    'max_depth': [ 3, 6, 9, 12],  # Maximum depth of trees
    'learning_rate': [0.01,0.02, 0.05, 0.1],  # Learning rate
    'subsample': [0.6, 0.7, 0.8, 1.0],  # Subsample ratio of training instances
    'colsample_bytree': [0.6, 0.8],  # Subsample ratio of columns
    'gamma': [0, 0.1, 0.2, 0.5],  # Minimum loss reduction
    'reg_alpha': [0, 1, 5, 10],  # L1 regularisation term
    'reg_lambda': [0, 1, 2, 3,4, 5]   # L2 regularisation term
}

# initialise XGBoost regression
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Create Random Search CV
random_search_xgb = RandomizedSearchCV(xgb_model,
                                       param_distributions=param_distributions_xgb,
                                       n_iter=15,  # Number of random combinations to try, change as needed
                                       cv=2,
                                       scoring='neg_mean_squared_error',
                                       verbose=2,
                                       random_state=42,
                                       n_jobs=-1)

# Fit random search CV
print("Starting RandomizedSearchCV for XGBoost...")
start_time_xgb = time.time()  # Start time

random_search_xgb.fit(x_train_scaled, y_train)

end_time_xgb = time.time()  # End time
print("RandomizedSearchCV for XGBoost fitting completed in:", end_time_xgb - start_time_xgb, "seconds")


# Get the best parameters
best_params_xgb = random_search_xgb.best_params_
print("Best parameters for XGBoost:", best_params_xgb)

# Get the best estimator (XGBoost model with best parameters)
best_xgb_model = random_search_xgb.best_estimator_
print("Best estimator:", best_xgb_model)

# 5. Multi-Target Model Evaluation (using the best estimator) and convert to DF
y_pred_xgb = best_xgb_model.predict(x_test_scaled)
y_pred_xgb_df = pd.DataFrame(y_pred_xgb, columns=['frp_pred', 'brightness_pred'], index=y_test.index)


# Calculate evaluation metrics for each target
rmse_frp_xgb = mean_squared_error(y_test['frp'], y_pred_xgb_df['frp_pred'])
rmse_brightness_xgb = mean_squared_error(y_test['brightness'], y_pred_xgb_df['brightness_pred'])
r2_frp_xgb = r2_score(y_test['frp'], y_pred_xgb_df['frp_pred'])
r2_brightness_xgb = r2_score(y_test['brightness'], y_pred_xgb_df['brightness_pred'])

# Print the evaluation metrics
print("--- Multi-Target XGBoost Regression Results (with RandomisedSearchCV) ---")
print("RMSE for FRP:", rmse_frp_xgb)
print("RMSE for Brightness:", rmse_brightness_xgb)
print("R-squared for FRP:", r2_frp_xgb)
print("R-squared for Brightness:", r2_brightness_xgb)

Starting RandomizedSearchCV for XGBoost...
Fitting 2 folds for each of 15 candidates, totalling 30 fits


/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


RandomizedSearchCV for XGBoost fitting completed in: 1312.0639352798462 seconds
Best parameters for XGBoost: {'subsample': 0.8, 'reg_lambda': 0, 'reg_alpha': 5, 'n_estimators': 2500, 'max_depth': 12, 'learning_rate': 0.05, 'gamma': 0.2, 'colsample_bytree': 0.6}
Best estimator: XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.6, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.2, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=12, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=2500, n_jobs=None,
             num_parallel_tree=None, ran

In [ ]:
# Parameter tuning for XGBoost 3
# Define parameter distribution
param_distributions_xgb = {
    'n_estimators': [500, 1000, 1500, 2500, 5000],  # Number of boosting rounds
    'max_depth': [ 9, 12, 20, 50],  # Maximum depth of trees
    'learning_rate': [0.01,0.02, 0.05, 0.1],  # Learning rate
    'subsample': [ 0.7, 0.8, 0.9, 1.0],  # Subsample ratio of training instances
    'colsample_bytree': [0.6, 0.8],  # Subsample ratio of columns
    'gamma': [0, 0.1, 0.2, 0.5, 0.75, 1.0],  # Minimum loss reduction
    'reg_alpha': [0, 1, 5, 10],  # L1 regularisation term
    'reg_lambda': [0, 1, 2, 3,4, 5]   # L2 regularisation term
}

# initialise XGBoost regression
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Create Random Search CV
random_search_xgb = RandomizedSearchCV(xgb_model,
                                       param_distributions=param_distributions_xgb,
                                       n_iter=8,  # Number of random combinations to try, change as needed
                                       cv=2,
                                       scoring='neg_mean_squared_error',
                                       verbose=2,
                                       random_state=42,
                                       n_jobs=-1)

# Fit random search CV
print("Starting RandomizedSearchCV for XGBoost...")
start_time_xgb = time.time()  # Start time

random_search_xgb.fit(x_train_scaled, y_train)

end_time_xgb = time.time()  # End time
print("RandomizedSearchCV for XGBoost fitting completed in:", end_time_xgb - start_time_xgb, "seconds")


# Get the best parameters
best_params_xgb = random_search_xgb.best_params_
print("Best parameters for XGBoost:", best_params_xgb)

# Get the best estimator (XGBoost model with best parameters)
best_xgb_model = random_search_xgb.best_estimator_
print("Best estimator:", best_xgb_model)

# 5. Multi-Target Model Evaluation (using the best estimator) and convert to DF
y_pred_xgb = best_xgb_model.predict(x_test_scaled)
y_pred_xgb_df = pd.DataFrame(y_pred_xgb, columns=['frp_pred', 'brightness_pred'], index=y_test.index)


# Calculate evaluation metrics for each target
rmse_frp_xgb = mean_squared_error(y_test['frp'], y_pred_xgb_df['frp_pred'])
rmse_brightness_xgb = mean_squared_error(y_test['brightness'], y_pred_xgb_df['brightness_pred'])
r2_frp_xgb = r2_score(y_test['frp'], y_pred_xgb_df['frp_pred'])
r2_brightness_xgb = r2_score(y_test['brightness'], y_pred_xgb_df['brightness_pred'])

# Print the evaluation metrics
print("--- Multi-Target XGBoost Regression Results (with RandomisedSearchCV) ---")
print("RMSE for FRP:", rmse_frp_xgb)
print("RMSE for Brightness:", rmse_brightness_xgb)
print("R-squared for FRP:", r2_frp_xgb)
print("R-squared for Brightness:", r2_brightness_xgb)

Starting RandomizedSearchCV for XGBoost...
Fitting 2 folds for each of 8 candidates, totalling 16 fits


/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


RandomizedSearchCV for XGBoost fitting completed in: 2805.0644297599792 seconds
Best parameters for XGBoost: {'subsample': 0.7, 'reg_lambda': 5, 'reg_alpha': 10, 'n_estimators': 2500, 'max_depth': 12, 'learning_rate': 0.01, 'gamma': 0, 'colsample_bytree': 0.6}
Best estimator: XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.6, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=12, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=2500, n_jobs=None,
             num_parallel_tree=None, random

In [ ]:
# XGBoost tuning 4
#   Define parameter distribution
param_distributions_xgb = {
    'n_estimators': [ 1500, 2500, 3000, 4000],  # Number of boosting rounds
    'max_depth': [9, 12, 15, 25, 50],  # Maximum depth of trees
    'learning_rate': [0.001, 0.005, 0.01],  # Learning rate
    'subsample': [0.7, 0.8, 0.9, 1.0],  # Subsample ratio of training instances
    'colsample_bytree': [0.5,0.6, 0.7, 0.8, 0.9, 1.0],  # Subsample ratio of columns
    'gamma': [0, 0.1, 0.2, 0.5, 0.75, 1.0],  # Minimum loss reduction
    'reg_alpha': [0, 0.1, 1, 5, 10, 20, 50],  # L1 regularisation term
    'reg_lambda': [0, 0.1, 1, 5, 10, 20, 50],   # L2 regularisation term
    'min_child_weight': [1, 3, 5, 7]  # Minimum sum of instance weight (hessian) needed in a child
}

#   initialise XGBoost regression
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

#   Create Random Search CV
random_search_xgb = RandomizedSearchCV(xgb_model,
                                       param_distributions=param_distributions_xgb,
                                       n_iter=6,  # Number of random combinations to try, change as needed
                                       cv=2,
                                       scoring='neg_mean_squared_error',
                                       verbose=2,
                                       random_state=42,
                                       n_jobs=-1)

#   Fit random search CV
print("Starting RandomizedSearchCV for XGBoost...")
start_time_xgb = time.time()  # Start time

random_search_xgb.fit(x_train_scaled, y_train)

end_time_xgb = time.time()  # End time
print("RandomizedSearchCV for XGBoost fitting completed in:", end_time_xgb - start_time_xgb, "seconds")

#   Get the best parameters
best_params_xgb = random_search_xgb.best_params_
print("Best parameters for XGBoost:", best_params_xgb)

#   Get the best estimator (XGBoost model with best parameters)
best_xgb_model = random_search_xgb.best_estimator_
print("Best estimator:", best_xgb_model)

#   5. Multi-Target Model Evaluation (using the best estimator) and convert to DF
y_pred_xgb = best_xgb_model.predict(x_test_scaled)
y_pred_xgb_df = pd.DataFrame(y_pred_xgb, columns=['frp_pred', 'brightness_pred'], index=y_test.index)

#   Calculate metrics for each target
rmse_frp_xgb = mean_squared_error(y_test['frp'], y_pred_xgb_df['frp_pred'])
rmse_brightness_xgb = mean_squared_error(y_test['brightness'], y_pred_xgb_df['brightness_pred'])
r2_frp_xgb = r2_score(y_test['frp'], y_pred_xgb_df['frp_pred'])
r2_brightness_xgb = r2_score(y_test['brightness'], y_pred_xgb_df['brightness_pred'])

#   Print the evaluation metrics
print("--- Multi-Target XGBoost Regression Results tuning #4 (with RandomizedSearchCV) ---")
print("RMSE for FRP:", rmse_frp_xgb)
print("RMSE for Brightness:", rmse_brightness_xgb)
print("R-squared for FRP:", r2_frp_xgb)
print("R-squared for Brightness:", r2_brightness_xgb)

Starting RandomizedSearchCV for XGBoost...
Fitting 2 folds for each of 6 candidates, totalling 12 fits
RandomizedSearchCV for XGBoost fitting completed in: 2914.2217288017273 seconds
Best parameters for XGBoost: {'subsample': 0.9, 'reg_lambda': 1, 'reg_alpha': 1, 'n_estimators': 3000, 'min_child_weight': 5, 'max_depth': 15, 'learning_rate': 0.005, 'gamma': 1.0, 'colsample_bytree': 0.5}
Best estimator: XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.5, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=1.0, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.005, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=15, max_leaves=None,
             min_child_weight=5, missing=nan, monotone

In [ ]:
# ANN1
# Neural network - use T4 or L4

# Get the number of input features
num_features = x_train_scaled.shape[1]

# Design the neural network architecture
model = keras.Sequential([
    layers.Input(shape=(num_features,)),  # Input layer (30 features)
    layers.Dense(256, activation='relu'),  # Hidden layer 1
    layers.Dense(128, activation='relu'),  # Hidden layer 2
    layers.Dense(64, activation='relu'),   # Hidden layer 3
    layers.Dense(2, activation='linear')   # Output layer (2 neurons for FRP and brightness)
])

# Compile the model
model.compile(optimizer='adam',
              loss='mean_squared_error',  # Loss function: Mean squared error for regression
              metrics=['mean_absolute_error'])  # Metrics: Mean absolute error is a good metric for regression

# Model Summary
model.summary()

# Model Training
print("Starting Neural Network training...")
start_time_nn = time.time()  # Start time

history = model.fit(x_train_scaled, y_train,  # Train on scaled training data
                    epochs=20,             # Number of training epochs
                    batch_size=64,          # Batch size
                    validation_data=(x_test_scaled, y_test),  # Evaluate on test data during training
                    verbose=1)              # Show training progress

end_time_nn = time.time()  # End time
print("Neural Network training completed in:", end_time_nn - start_time_nn, "seconds")

#Model Evaluation
# Make predictions on the test set and convert to DF
y_pred_nn = model.predict(x_test_scaled)
y_pred_nn_df = pd.DataFrame(y_pred_nn, columns=['frp_pred', 'brightness_pred'], index=y_test.index)

# Calculate evaluation metrics for each target
rmse_frp_nn = mean_squared_error(y_test['frp'], y_pred_nn_df['frp_pred'])
rmse_brightness_nn = mean_squared_error(y_test['brightness'], y_pred_nn_df['brightness_pred'])
r2_frp_nn = r2_score(y_test['frp'], y_pred_nn_df['frp_pred'])
r2_brightness_nn = r2_score(y_test['brightness'], y_pred_nn_df['brightness_pred'])

# Print the evaluation metrics
print("--- Neural Network Regression Results ---")
print("RMSE for FRP:", rmse_frp_nn)
print("RMSE for Brightness:", rmse_brightness_nn)
print("R-squared for FRP:", r2_frp_nn)
print("R-squared for Brightness:", r2_brightness_nn)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 256)                 │           8,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 2)                   │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 49,986 (195.26 KB)

 Trainable params: 49,986 (195.26 KB)

 Non-trainable params: 0 (0.00 B)

Starting Neural Network training...
Epoch 1/20
5815/5815 ━━━━━━━━━━━━━━━━━━━━ 27s 4ms/step - loss: 7103.7725 - mean_absolute_error: 25.2804 - val_loss: 3700.1714 - val_mean_absolute_error: 15.7488
Epoch 2/20
5815/5815 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - loss: 4171.4644 - mean_absolute_error: 16.1505 - val_loss: 3783.4614 - val_mean_absolute_error: 14.7787
Epoch 3/20
5815/5815 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - loss: 3655.3149 - mean_absolute_error: 15.7098 - val_loss: 3601.7437 - val_mean_absolute_error: 15.7385
Epoch 4/20
5815/5815 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - loss: 4008.1169 - mean_absolute_error: 15.6107 - val_loss: 3531.7507 - val_mean_absolute_error: 14.4092
Epoch 5/20
5815/5815 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - loss: 3867.0378 - mean_absolute_error: 15.2053 - val_loss: 3563.1548 - val_mean_absolute_error: 16.2904
Epoch 6/20
5815/5815 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - loss: 3633.9468 - mean_absolute_error: 15.1188 - val_loss: 3478.0920 - val_mean_absolute_error: 14.830

In [ ]:
#ANN2
# Get the number of input features
num_features = x_train_scaled.shape[1]

# Design the neural network architecture (shallower example)
model = keras.Sequential([
    layers.Input(shape=(num_features,)),  # Input layer (30 features)
    layers.Dense(128, activation='relu'),  # Hidden layer 1
    layers.Dense(64, activation='relu'),   # Hidden layer 2
    layers.Dense(2, activation='linear')    # Output layer
])

# Compile the model
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mean_absolute_error'])

# Model Summary
model.summary()

#  Model Training
print("Starting Neural Network training...")
start_time_nn = time.time()  # Start time

# Implement Early Stopping
early_stopping = EarlyStopping(monitor='val_loss',  # Monitor validation loss
                              patience=10,         # Stop if no improvement for 10 epochs
                              restore_best_weights=True)  # Restore best model weights

history = model.fit(x_train_scaled, y_train,  # Train on scaled training data
                    epochs=50,             # Number of training epochs
                    batch_size=32,          # Batch size
                    validation_data=(x_test_scaled, y_test),  # Evaluate on test data during training
                    verbose=1,
                    callbacks=[early_stopping])  # Add early stopping

end_time_nn = time.time()  # End time
print("Neural Network training completed in:", end_time_nn - start_time_nn, "seconds")

# Model Evaluation
# Make predictions on the test set and convert to DF
y_pred_nn = model.predict(x_test_scaled)
y_pred_nn_df = pd.DataFrame(y_pred_nn, columns=['frp_pred', 'brightness_pred'], index=y_test.index)

# Calculate evaluation metrics for each target
rmse_frp_nn = mean_squared_error(y_test['frp'], y_pred_nn_df['frp_pred'])  # RMSE for FRP
rmse_brightness_nn = mean_squared_error(y_test['brightness'],  y_pred_nn_df['brightness_pred'])  # RMSE for brightness
r2_frp_nn = r2_score(y_test['frp'], y_pred_nn_df['frp_pred'])  # R-squared for FRP
r2_brightness_nn = r2_score(y_test['brightness'], y_pred_nn_df['brightness_pred'])  # R-squared for brightness

# Print the evaluation metrics
print("--- Neural Network Regression Results ---")
print("RMSE for FRP:", rmse_frp_nn)
print("RMSE for Brightness:", rmse_brightness_nn)
print("R-squared for FRP:", r2_frp_nn)
print("R-squared for Brightness:", r2_brightness_nn)

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_19 (Dense)                     │ (None, 128)                 │           4,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_20 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ (None, 2)                   │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,738 (49.76 KB)

 Trainable params: 12,738 (49.76 KB)

 Non-trainable params: 0 (0.00 B)

Starting Neural Network training...
Epoch 1/50
11630/11630 ━━━━━━━━━━━━━━━━━━━━ 30s 2ms/step - loss: 6907.0723 - mean_absolute_error: 27.4244 - val_loss: 3771.9851 - val_mean_absolute_error: 14.8387
Epoch 2/50
11630/11630 ━━━━━━━━━━━━━━━━━━━━ 27s 2ms/step - loss: 3581.5017 - mean_absolute_error: 15.7022 - val_loss: 3674.6428 - val_mean_absolute_error: 15.5087
Epoch 3/50
11630/11630 ━━━━━━━━━━━━━━━━━━━━ 28s 2ms/step - loss: 3619.7766 - mean_absolute_error: 15.6273 - val_loss: 3633.1558 - val_mean_absolute_error: 15.2422
Epoch 4/50
11630/11630 ━━━━━━━━━━━━━━━━━━━━ 27s 2ms/step - loss: 3397.3018 - mean_absolute_error: 15.5166 - val_loss: 3620.4045 - val_mean_absolute_error: 15.7942
Epoch 5/50
11630/11630 ━━━━━━━━━━━━━━━━━━━━ 26s 2ms/step - loss: 3482.4424 - mean_absolute_error: 15.3346 - val_loss: 3596.8469 - val_mean_absolute_error: 14.5751
Epoch 6/50
11630/11630 ━━━━━━━━━━━━━━━━━━━━ 27s 2ms/step - loss: 3392.3762 - mean_absolute_error: 15.1825 - val_loss: 3695.0347 - val_mean_absolute_e

In [ ]:
#ANN3
num_features = x_train_scaled.shape[1]

# Design the neural network architecture (deeper example)
model = keras.Sequential([
    layers.Input(shape=(num_features,)),  # Input layer (30 features)
    layers.Dense(512, activation='relu'),  # Hidden layer 1
    layers.Dense(256, activation='relu'),  # Hidden layer 2
    layers.Dense(128, activation='relu'),  # Hidden layer 3
    layers.Dense(64, activation='relu'),   # Hidden layer 4
    layers.Dense(2, activation='linear')    # Output layer
])

# Compile the model
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mean_absolute_error'])

# Model Summary
model.summary()

#  Model Training
print("Starting Neural Network training...")
start_time_nn = time.time()  # Start time

# Implement Early Stopping
early_stopping = EarlyStopping(monitor='val_loss',  # Monitor validation loss
                              patience=10,         # Stop if no improvement for 10 epochs
                              restore_best_weights=True)  # Restore best model weights

history = model.fit(x_train_scaled, y_train,  # Train on scaled training data
                    epochs=50,             # Number of training epochs
                    batch_size=32,          # Batch size
                    validation_data=(x_test_scaled, y_test),  # Evaluate on test data during training
                    verbose=1,
                    callbacks=[early_stopping])  # Add early stopping

end_time_nn = time.time()  # End time
print("Neural Network training completed in:", end_time_nn - start_time_nn, "seconds")

# Model Evaluation
# Make predictions on the test set and convert
y_pred_nn = model.predict(x_test_scaled)
y_pred_nn_df = pd.DataFrame(y_pred_nn, columns=['frp_pred', 'brightness_pred'], index=y_test.index)

# Calculate evaluation metrics for each target
rmse_frp_nn = mean_squared_error(y_test['frp'], y_pred_nn_df['frp_pred'])
rmse_brightness_nn = mean_squared_error(y_test['brightness'], y_pred_nn_df['brightness_pred'])
r2_frp_nn = r2_score(y_test['frp'], y_pred_nn_df['frp_pred'])
r2_brightness_nn = r2_score(y_test['brightness'], y_pred_nn_df['brightness_pred'])

# Print the evaluation metrics
print("--- Neural Network Regression Results ---")
print("RMSE for FRP:", rmse_frp_nn)
print("RMSE for Brightness:", rmse_brightness_nn)
print("R-squared for FRP:", r2_frp_nn)
print("R-squared for Brightness:", r2_brightness_nn)


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_22 (Dense)                     │ (None, 512)                 │          17,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_23 (Dense)                     │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_24 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_25 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_26 (Dense)                     │ (None, 2)                   │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 190,018 (742.26 KB)

 Trainable params: 190,018 (742.26 KB)

 Non-trainable params: 0 (0.00 B)

Starting Neural Network training...
Epoch 1/50
11630/11630 ━━━━━━━━━━━━━━━━━━━━ 39s 3ms/step - loss: 5500.6206 - mean_absolute_error: 20.8904 - val_loss: 3975.7888 - val_mean_absolute_error: 17.3021
Epoch 2/50
11630/11630 ━━━━━━━━━━━━━━━━━━━━ 28s 2ms/step - loss: 3933.0986 - mean_absolute_error: 15.8165 - val_loss: 3708.0759 - val_mean_absolute_error: 15.1592
Epoch 3/50
11630/11630 ━━━━━━━━━━━━━━━━━━━━ 28s 2ms/step - loss: 3961.0320 - mean_absolute_error: 15.3064 - val_loss: 3771.4626 - val_mean_absolute_error: 13.7707
Epoch 4/50
11630/11630 ━━━━━━━━━━━━━━━━━━━━ 28s 2ms/step - loss: 3649.6255 - mean_absolute_error: 14.9520 - val_loss: 3532.3086 - val_mean_absolute_error: 14.9015
Epoch 5/50
11630/11630 ━━━━━━━━━━━━━━━━━━━━ 28s 2ms/step - loss: 3785.2253 - mean_absolute_error: 14.9962 - val_loss: 3466.1064 - val_mean_absolute_error: 14.0726
Epoch 6/50
11630/11630 ━━━━━━━━━━━━━━━━━━━━ 28s 2ms/step - loss: 3771.6553 - mean_absolute_error: 14.8887 - val_loss: 3466.1890 - val_mean_absolute_e

In [ ]:
#ANN4
num_features = x_train_scaled.shape[1]

# Design the neural network architecture (shallower example)
model = keras.Sequential([
    layers.Input(shape=(num_features,)),  # Input layer (30 features)
    layers.Dense(256, kernel_initializer='lecun_normal', activation='elu'),  # Hidden layer 1
    layers.Dense(128, kernel_initializer='lecun_normal', activation='elu'),  # Hidden layer 2
    layers.Dense(64, kernel_initializer='lecun_normal', activation='elu'),  # Hidden layer 3
    layers.Dense(2, activation='linear')    # Output layer (2 neurons for FRP and brightness)
])

# Compile the model
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mean_absolute_error'])

# Model Summary
model.summary()

# 5. Model Training
print("Starting Neural Network training...")
start_time_nn = time.time()  # Start time

# Implement Early Stopping
early_stopping = EarlyStopping(monitor='val_loss',  # Monitor validation loss
                              patience=10,         # Stop if no improvement for 10 epochs
                              restore_best_weights=True)  # Restore best model weights

history = model.fit(x_train_scaled, y_train,  # Train on scaled training data
                    epochs=50,             # Number of training epochs (adjust!)
                    batch_size=32,          # Batch size (adjust!)
                    validation_data=(x_test_scaled, y_test),  # Evaluate on test data during training
                    verbose=1,
                    callbacks=[early_stopping])  # Add early stopping

end_time_nn = time.time()  # End time
print("Neural Network training completed in:", end_time_nn - start_time_nn, "seconds")

# Model Evaluation
# Make predictions on the test set, convert to dataframe
y_pred_nn = model.predict(x_test_scaled)
y_pred_nn_df = pd.DataFrame(y_pred_nn, columns=['frp_pred', 'brightness_pred'],  index=y_test.index)

# Calculate evaluation metrics for each target
rmse_frp_nn = mean_squared_error(y_test['frp'], y_pred_nn_df['frp_pred'])
rmse_brightness_nn = mean_squared_error(y_test['brightness'], y_pred_nn_df['brightness_pred'])
r2_frp_nn = r2_score(y_test['frp'], y_pred_nn_df['frp_pred'])
r2_brightness_nn = r2_score(y_test['brightness'], y_pred_nn_df['brightness_pred'])

# Print the evaluation metrics
print("--- Neural Network Regression Results ---")
print("RMSE for FRP:", rmse_frp_nn)
print("RMSE for Brightness:", rmse_brightness_nn)
print("R-squared for FRP:", r2_frp_nn)
print("R-squared for Brightness:", r2_brightness_nn)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 256)                 │           8,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 2)                   │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 49,986 (195.26 KB)

 Trainable params: 49,986 (195.26 KB)

 Non-trainable params: 0 (0.00 B)

Starting Neural Network training...
Epoch 1/50
11630/11630 ━━━━━━━━━━━━━━━━━━━━ 34s 3ms/step - loss: 5444.1470 - mean_absolute_error: 22.7740 - val_loss: 3769.7126 - val_mean_absolute_error: 15.1532
Epoch 2/50
11630/11630 ━━━━━━━━━━━━━━━━━━━━ 32s 3ms/step - loss: 4554.8525 - mean_absolute_error: 16.0730 - val_loss: 3710.1472 - val_mean_absolute_error: 14.3503
Epoch 3/50
11630/11630 ━━━━━━━━━━━━━━━━━━━━ 32s 3ms/step - loss: 3668.7480 - mean_absolute_error: 15.1043 - val_loss: 3541.7043 - val_mean_absolute_error: 14.2391
Epoch 4/50
11630/11630 ━━━━━━━━━━━━━━━━━━━━ 32s 3ms/step - loss: 3585.6123 - mean_absolute_error: 14.6425 - val_loss: 3621.3696 - val_mean_absolute_error: 14.2266
Epoch 5/50
11630/11630 ━━━━━━━━━━━━━━━━━━━━ 34s 3ms/step - loss: 3836.8225 - mean_absolute_error: 14.5384 - val_loss: 3497.0432 - val_mean_absolute_error: 14.1618
Epoch 6/50
11630/11630 ━━━━━━━━━━━━━━━━━━━━ 34s 3ms/step - loss: 3571.8137 - mean_absolute_error: 14.4556 - val_loss: 3455.7646 - val_mean_absolute_e

In [ ]:
# ANN5

# Get the number of input features
num_features = x_train_scaled.shape[1]

# Design the neural network architecture (ANN 5 configuration)
model = keras.Sequential([
    layers.Input(shape=(num_features,)),  # Input layer (30 features)
    layers.Dense(256, kernel_initializer='lecun_normal', activation='tanh'),  # Hidden layer 1 (lecun, tanh)
    layers.Dropout(0.2),  # Dropout layer (dropout_rate = 0.2)
    layers.Dense(128, kernel_initializer='lecun_normal', activation='tanh'),  # Hidden layer 2 (lecun, tanh)
    layers.Dropout(0.2),  # Dropout layer (dropout_rate = 0.2)
    layers.Dense(64, kernel_initializer='lecun_normal', activation='tanh'),  # Hidden layer 3 (lecun, tanh)
    layers.Dropout(0.2),  # Dropout layer (dropout_rate = 0.2)
    layers.Dense(2, activation='linear')    # Output layer (2 neurons for FRP and brightness)
])

# Compile the model
model.compile(optimizer='rmsprop',  # Optimizer: RMSprop
              loss='mean_squared_error',  # Loss function: Mean squared error for regression
              metrics=['mean_absolute_error'])  # Metrics: Mean absolute error is a good metric for regression

# Model Summary (useful for visualizing the architecture)
model.summary()

#  Model Training
print("Starting Neural Network training (ANN 5)...")
start_time_nn = time.time()  # Start time

# Implement Early Stopping
early_stopping = EarlyStopping(monitor='val_loss',  # Monitor validation loss
                              patience=10,         # Stop if no improvement for 10 epochs
                              restore_best_weights=True)  # Restore best model weights

history = model.fit(x_train_scaled, y_train,  # Train on scaled training data
                    epochs=50,             # Number of training epochs
                    batch_size=64,          # Batch size
                    validation_data=(x_test_scaled, y_test),  # Evaluate on test data during training
                    verbose=1,
                    callbacks=[early_stopping])  # Add early stopping

end_time_nn = time.time()  # End time
print("Neural Network training completed in:", end_time_nn - start_time_nn, "seconds")

#  Model Evaluation
# Make predictions on the test set
y_pred_nn = model.predict(x_test_scaled)
y_pred_nn_df = pd.DataFrame(y_pred_nn, columns=['frp_pred', 'brightness_pred'], index=y_test.index)

# Calculate evaluation metrics for each target
rmse_frp_nn = mean_squared_error(y_test['frp'], y_pred_nn_df['frp_pred'])
rmse_brightness_nn = mean_squared_error(y_test['brightness'], y_pred_nn_df['brightness_pred'])
r2_frp_nn = r2_score(y_test['frp'], y_pred_nn_df['frp_pred'])
r2_brightness_nn = r2_score(y_test['brightness'], y_pred_nn_df['frp_pred'])

# Print the evaluation metrics
print("--- Neural Network Regression Results (ANN 5) ---")
print("RMSE for FRP:", rmse_frp_nn)
print("RMSE for Brightness:", rmse_brightness_nn)
print("R-squared for FRP:", r2_frp_nn)
print("R-squared for Brightness:", r2_brightness_nn)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                      │ (None, 256)                 │           8,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 2)                   │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 49,986 (195.26 KB)

 Trainable params: 49,986 (195.26 KB)

 Non-trainable params: 0 (0.00 B)

Starting Neural Network training (ANN 5)...
Epoch 1/50
5815/5815 ━━━━━━━━━━━━━━━━━━━━ 23s 4ms/step - loss: 38423.7383 - mean_absolute_error: 130.6590 - val_loss: 19121.6289 - val_mean_absolute_error: 54.6344
Epoch 2/50
5815/5815 ━━━━━━━━━━━━━━━━━━━━ 22s 4ms/step - loss: 19556.1523 - mean_absolute_error: 54.8232 - val_loss: 18563.4707 - val_mean_absolute_error: 53.6068
Epoch 3/50
5815/5815 ━━━━━━━━━━━━━━━━━━━━ 22s 4ms/step - loss: 18826.7344 - mean_absolute_error: 53.9529 - val_loss: 18020.6074 - val_mean_absolute_error: 52.4323
Epoch 4/50
5815/5815 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - loss: 18068.6875 - mean_absolute_error: 52.8209 - val_loss: 17495.4297 - val_mean_absolute_error: 52.2774
Epoch 5/50
5815/5815 ━━━━━━━━━━━━━━━━━━━━ 22s 4ms/step - loss: 17952.8320 - mean_absolute_error: 52.3462 - val_loss: 16983.9805 - val_mean_absolute_error: 51.3148
Epoch 6/50
5815/5815 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - loss: 17434.3730 - mean_absolute_error: 52.0505 - val_loss: 16488.9844 - val_mean_a

In [ ]:
#ANN6
num_features = x_train_scaled.shape[1]
# Design the neural network architecture (ANN 6 configuration)
model_ann6 = keras.Sequential([
    layers.Input(shape=(num_features,)),  # Input layer (30 features)
    layers.Dense(512, kernel_initializer='he_normal', activation='relu'),  # Hidden layer 1 (ReLU, more neurons)
    layers.Dropout(0.1),  # Dropout layer (dropout_rate = 0.1)
    layers.Dense(256, kernel_initializer='he_normal', activation='relu'),  # Hidden layer 2 (ReLU)
    layers.Dropout(0.1),  # Dropout layer (dropout_rate = 0.1)
    layers.Dense(128, kernel_initializer='he_normal', activation='relu'),  # Hidden layer 3 (ReLU)
    layers.Dense(64, kernel_initializer='he_normal', activation='relu'),   # Hidden layer 4 (ReLU)
    layers.Dense(2, activation='linear')    # Output layer (2 neurons for FRP and brightness)
])

# Compile the model
model_ann6.compile(optimizer='adam',
                  loss='mean_squared_error',
                  metrics=['mean_absolute_error'])

# Model Summary
print("--- ANN 6 Architecture ---")
model_ann6.summary()

#  Model Training (ANN 6)
print("Starting Neural Network training (ANN 6)...")
start_time_ann6 = time.time()  # Start time

# Implement Early Stopping
early_stopping = EarlyStopping(monitor='val_loss',  # Monitor validation loss
                              patience=10,         # Stop if no improvement for 10 epochs
                              restore_best_weights=True)  # Restore best model weights

history_ann6 = model_ann6.fit(x_train_scaled, y_train,  # Train on scaled training data
                    epochs=50,             # Number of training epochs
                    batch_size=32,          # Batch size
                    validation_data=(x_test_scaled, y_test),  # Evaluate on test data during training
                    verbose=1,
                    callbacks=[early_stopping])  # Add early stopping

end_time_ann6 = time.time()  # End time
print("Neural Network training completed in:", end_time_ann6 - start_time_ann6, "seconds")

#  Model Evaluation (ANN 6)
# Make predictions on the test set
y_pred_ann6 = model_ann6.predict(x_test_scaled)
y_pred_ann6_df = pd.DataFrame(y_pred_ann6, columns=['frp_pred', 'brightness_pred'], index=y_test.index)

# Calculate evaluation metrics for each target
rmse_frp_ann6 = mean_squared_error(y_test['frp'], y_pred_ann6_df['frp_pred'])
rmse_brightness_ann6 = mean_squared_error(y_test['brightness'],  y_pred_ann6_df['brightness_pred'])
r2_frp_ann6 = r2_score(y_test['frp'], y_pred_ann6_df['frp_pred'])
r2_brightness_ann6 = r2_score(y_test['brightness'], y_pred_ann6_df['brightness_pred'])

# Print the evaluation metrics
print("--- Neural Network Regression Results (ANN 6) ---")
print("RMSE for FRP:", rmse_frp_ann6)
print("RMSE for Brightness:", rmse_brightness_ann6)
print("R-squared for FRP:", r2_frp_ann6)
print("R-squared for Brightness:", r2_brightness_ann6)


--- ANN 6 Architecture ---


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                      │ (None, 512)                 │          17,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 2)                   │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 190,018 (742.26 KB)

 Trainable params: 190,018 (742.26 KB)

 Non-trainable params: 0 (0.00 B)

Starting Neural Network training (ANN 6)...
Epoch 1/50
11630/11630 ━━━━━━━━━━━━━━━━━━━━ 51s 4ms/step - loss: 5667.2188 - mean_absolute_error: 22.6946 - val_loss: 3888.3179 - val_mean_absolute_error: 19.9188
Epoch 2/50
11630/11630 ━━━━━━━━━━━━━━━━━━━━ 51s 4ms/step - loss: 3694.2854 - mean_absolute_error: 17.2474 - val_loss: 4118.2739 - val_mean_absolute_error: 23.7426
Epoch 3/50
11630/11630 ━━━━━━━━━━━━━━━━━━━━ 51s 4ms/step - loss: 4059.1863 - mean_absolute_error: 16.6671 - val_loss: 4100.7871 - val_mean_absolute_error: 24.6211
Epoch 4/50
11630/11630 ━━━━━━━━━━━━━━━━━━━━ 51s 4ms/step - loss: 3625.0325 - mean_absolute_error: 16.0563 - val_loss: 3839.0669 - val_mean_absolute_error: 20.3766
Epoch 5/50
11630/11630 ━━━━━━━━━━━━━━━━━━━━ 51s 4ms/step - loss: 3854.4104 - mean_absolute_error: 15.9014 - val_loss: 3942.1077 - val_mean_absolute_error: 22.0858
Epoch 6/50
11630/11630 ━━━━━━━━━━━━━━━━━━━━ 49s 4ms/step - loss: 3404.5620 - mean_absolute_error: 15.7597 - val_loss: 3914.9724 - val_mean_ab